In [84]:
# from google.colab import drive
# drive.mount('/content/drive')

In [15]:
# !pip install -q transformers

In [16]:
# !pip install -q accelerate

In [5]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
import torch
from transformers import GPTJForCausalLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", revision="float16", low_cpu_mem_usage=True)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

Downloading:   0%|          | 0.00/836 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.3G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

## Generate Prompts

In [13]:
data_path = "/kaggle/input/semeval-2023-task-10/train_all_tasks.csv"

In [14]:
import pandas as pd
data = pd.read_csv(data_path)
data = data[data["label_category"]!="none"]

In [53]:
target_column = "label_category"

In [17]:
label_category_map = {
    "1. threats, plans to harm and incitement":"Threat",
    "2. derogation":"Derogation",
    "3. animosity":"Animosity",
    "4. prejudiced discussions":"Prejudiced discussion",
    "Threat":"1. threats, plans to harm and incitement",
    "Derogation":"2. derogation",
    "Animosity":"3. animosity",
    "Prejudiced discussion":"4. prejudiced discussions"
}

In [57]:
labels = data[target_column].unique()
labels

array(['2. derogation', '1. threats, plans to harm and incitement',
       '3. animosity', '4. prejudiced discussions'], dtype=object)

In [58]:
import random
# function returns two random text samples to use them as GPT-J prompt
# def get_three_random_samples(data_frame, target_column):
#     # get two random samples from `data file`
#     s1, s2, s3 = random.sample(range(0, len(data)), 3)

#     # get text labels
#     d = data_frame.iloc[[s1, s2, s3]]
#     return list(zip(d['text'].values, d[target_column].values))

def get_three_random_samples(data_frame, target_column, label, n=3):
    # get two random samples from `data file`
    
    d = data_frame[data_frame[target_column]==label]
    s1, s2, s3 = random.sample(range(0, len(d)), n)

    # get text labels
    d = d.iloc[[s1, s2, s3]]
    return list(zip(d['text'].values, d[target_column].values))

samples = get_three_random_samples(data, target_column, labels[0])

In [61]:
# def get_prompt(samples):
#     # define a function that takes as input three samples and generates the prompt
#     # that we should pass to the GPT-3 language model for completion.
#     description = "Each item in the following list contains a sexist tweet and the respective category. Category is one of 'Threat', 'Derogation', 'Animosity', or 'Prejudiced discussion'"
    
#     prompt = f"{description}\n"
#     for text, label in samples:
#         prompt += f"Tweet: {text} (Sentiment: {label_category_map[label]})\n"
    
#     return prompt

def get_prompt(samples, label):
    prompt = f"These are examples of {label_category_map[label]} tweets:\n"
    for text, label in samples:
        prompt += f"* {text}\n"
    
    return prompt

prompt = get_prompt(samples, labels[0])
print(prompt)

These are examples of Derogation tweets:
* All of those SJW purple haired foids over IT have 5'2" balding curry janitor bfs and that's a fact.
* Women only,girls only. Imagine if it was Men only. They be crying oppression and anti wymyn. Just cunts being cunts. This are the types where you just ignore them and act like they don't even exists. Too bad for this guy.
* Ive spent time in Sydney. They have legal prostitution there so the women have to be slightly less cunty to compete with the professionals. Its better than the west.



In [62]:
def generate_text(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    generated_ids = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=500)
    generated_text = tokenizer.decode(generated_ids[0])
    return generated_text

new_text = generate_text(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [63]:
print(prompt)

These are examples of Derogation tweets:
* All of those SJW purple haired foids over IT have 5'2" balding curry janitor bfs and that's a fact.
* Women only,girls only. Imagine if it was Men only. They be crying oppression and anti wymyn. Just cunts being cunts. This are the types where you just ignore them and act like they don't even exists. Too bad for this guy.
* Ive spent time in Sydney. They have legal prostitution there so the women have to be slightly less cunty to compete with the professionals. Its better than the west.



In [66]:
prompts = [
    get_prompt(samples, labels[0]),
    get_prompt(samples, labels[0])
]

In [68]:
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

In [82]:
def generate_text_in_batch(prompts):
    encoding = tokenizer(prompts, padding=True, return_tensors='pt').to(device)
    with torch.no_grad():
        generated_ids = model.generate(**encoding, do_sample=True, temperature=0.9, max_length=300, pad_token_id=tokenizer.eos_token_id)
    generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    
    return generated_texts

generated_texts = generate_text_in_batch(prompts)

In [ ]:
from tqdm import tqdm
import json

texts = {}

nbatch = 2500
batch_size = 4

for label in labels:
    texts[label] = []
    
    
for i in tqdm(range(nbatch), total=nbatch):
    for label in labels:
        prompts = []
        for b in range(batch_size):
            prompts.append(get_prompt(samples, label))
        generated_texts = generate_text_in_batch(prompts)
        texts[label].append(generated_texts)
    
    if i%10==0 and i!=0:
        with open(f'gpt_generated_texts_{i}.json', 'w') as outfile:
            json.dump(texts, outfile)


  6%|▋         | 159/2500 [2:53:57<42:37:33, 65.55s/it]

In [ ]:
"DONE"